In [1]:
import requests
import pandas as pd

In [ ]:
def call_text_search_api(fieldmask, textquery, api_key):

    url = 'https://places.googleapis.com/v1/places:searchText'

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': fieldmask
    }

    data = {
        'textQuery': textquery
    }

    response = requests.post(url, headers=headers, json=data)   # post accepts a json request body which allows for more parameters to be specified
    result = response.json()

    places_list = result.get('places', [])

    df = pd.json_normalize(places_list)

    return df

In [4]:
def call_nearby_search_api(fieldmask, loc_restriction, included_types, api_key):
    
    url = 'https://places.googleapis.com/v1/places:searchNearby'

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': fieldmask
    }

    data = {
        'locationRestriction': loc_restriction,
        'includedTypes': included_types
    }

    response = requests.post(url, headers=headers, json=data)
    result = response.json()

    places_list = result.get('places', [])

    df = pd.json_normalize(places_list)

    return df

In [ ]:
def call_place_details_api(fieldmask, place_id, api_key):

    url = 'https://places.googleapis.com/v1/places/' + place_id

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': fieldmask
    }

    response = requests.get(url, headers=headers)   # get retrieves a single resource based on a unique id
    result = response.json()

    df = pd.json_normalize(result)

    return df

In [ ]:
api_key = 'replace with api key here'

In [ ]:
# text search api (useful for retrieving place id)

fieldmask_1_1 = '*'   # retrieves all fields
textquery_1_1 = 'hawker stalls in Newton Food Centre Singapore'

test_1_1 = call_text_search_api(fieldmask_1_1, textquery_1_1, api_key)

In [55]:
test_1_1

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,postalAddress.postalCode,postalAddress.addressLines,servesLunch,servesDinner,liveMusic,menuForChildren,goodForWatchingSports,containingPlaces,regularOpeningHours.nextOpenTime,currentOpeningHours.nextOpenTime
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,229495,[500 Clemenceau Ave N],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJxcbG7-sZ2jERiyhLwTAA5i8,ChIJxcbG7-sZ2jERiyhLwTAA5i8,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-19 Newton Food Centr...","[{'longText': '01-19', 'shortText': '01-19', '...",4.7,https://maps.google.com/?cid=34514463738274223...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,229495,"[500 Clemenceau Ave N, #01-19 Newton Food Centre]",True,True,False,False,False,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ'...,2025-11-03T08:30:00Z,2025-11-03T08:30:00Z


In [61]:
fieldmask_1_2 = '*'   # retrieves all fields
textquery_1_2 = 'kwang kee teochew fish porridge in newton food centre'

test_1_2 = call_text_search_api(fieldmask_1_2, textquery_1_2, api_key)

In [62]:
test_1_2

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,websiteUri,utcOffsetMinutes,adrFormatAddress,...,internationalPhoneNumber,delivery,curbsidePickup,reservable,outdoorSeating,paymentOptions.acceptsDebitCards,paymentOptions.acceptsCashOnly,paymentOptions.acceptsNfc,accessibilityOptions.wheelchairAccessibleParking,accessibilityOptions.wheelchairAccessibleEntrance
0,places/ChIJPbO-7-sZ2jER1zjzFT3nUyY,ChIJPbO-7-sZ2jER1zjzFT3nUyY,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-20, Singapore 229495","[{'longText': '#01-20', 'shortText': '#01-20',...",4.0,https://maps.google.com/?cid=27618052460555286...,https://food.grab.com/sg/en/restaurant/kwang-k...,480,"<span class=""street-address"">500 Clemenceau Av...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJTwkF-usZ2jEROLeehMbVn58,ChIJTwkF-usZ2jEROLeehMbVn58,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-37, Singapore 229495","[{'longText': '01-37', 'shortText': '01-37', '...",4.1,https://maps.google.com/?cid=11502147021932771...,https://www.facebook.com/SongKeeFishPorridge/,480,"<span class=""street-address"">500 Clemenceau Av...",...,+65 9828 2122,True,False,True,True,False,True,False,True,True


In [63]:
fieldmask_1_3 = '*'   # retrieves all fields
textquery_1_3 = 'hawker stalls in Ghim Moh Food Centre Singapore'

test_1_3 = call_text_search_api(fieldmask_1_3, textquery_1_3, api_key)

In [64]:
test_1_3

,name,id,types,nationalPhoneNumber,internationalPhoneNumber,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJN69e7Gga2jERsqinNIodaeg,ChIJN69e7Gga2jERsqinNIodaeg,"[market, restaurant, food, point_of_interest, ...",6225 5632,+65 6225 5632,"20 Ghim Moh Rd, Singapore 270020","[{'longText': '20', 'shortText': '20', 'types'...",4.3,https://maps.google.com/?cid=16746949168779995...,480,...,https://www.google.com/maps/place//data=!4m3!3...,SGD,1,SGD,10,Asia/Singapore,SG,en-US,270020,[20 Ghim Moh Rd]


In [65]:
fieldmask_1_4 = '*'   # retrieves all fields
textquery_1_4 = 'jin ji fish soup ban mian in ghim moh food centre'

test_1_4 = call_text_search_api(fieldmask_1_4, textquery_1_4, api_key)

In [66]:
test_1_4

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJ-YeheEsb2jERUm7WlYIylV4,ChIJ-YeheEsb2jERUm7WlYIylV4,"[restaurant, food, point_of_interest, establis...","01-13 Ghim Moh Rd, 20 Singapore 270020","[{'longText': '20', 'shortText': '20', 'types'...",4.3,https://maps.google.com/?cid=68154091475361091...,480,"<span class=""street-address"">01-13 Ghim Moh Rd...",OPERATIONAL,...,https://www.google.com/maps/place//data=!4m3!3...,SGD,1,SGD,10,Asia/Singapore,SG,en-US,270020,"[01-13 Ghim Moh Rd, 20]"


In [71]:
# nearby search api

fieldmask_2 = '*'
loc_restriction = {
  "circle": {
    "center": { "latitude": 1.3521, "longitude": 103.8198 },
    "radius": 100
  }
}
included_types = ['restaurant']   # 'hawker_stall' is not a supported type, need to use generic types (e.g. 'restaurant', 'cafe')

test_2 = call_nearby_search_api(fieldmask_2, loc_restriction, included_types, api_key)

In [72]:
test_2

,name,id,types,nationalPhoneNumber,internationalPhoneNumber,formattedAddress,addressComponents,rating,googleMapsUri,websiteUri,...,servesLunch,servesDinner,servesBrunch,liveMusic,servesCoffee,goodForChildren,goodForGroups,accessibilityOptions.wheelchairAccessibleParking,servesBreakfast,accessibilityOptions.wheelchairAccessibleEntrance
0,places/ChIJFYrmYr492jERedys0dSFTDI,ChIJFYrmYr492jERedys0dSFTDI,"[vegetarian_restaurant, restaurant, food, poin...",8771 5803,+65 8771 5803,"477, #01-192, Singapore 523477","[{'longText': '477', 'languageCode': 'en'}, {'...",3.8,https://maps.google.com/?cid=36244189492237343...,https://www.instagram.com/p/DPVK2d9kz1Z/?utm_s...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJAQo3aDcR2jERRBG3b1Ak1Hs,ChIJAQo3aDcR2jERRBG3b1Ak1Hs,"[restaurant, food, point_of_interest, establis...",8833 0033,+65 8833 0033,Singapore 123456,"[{'longText': 'Singapore', 'shortText': 'Singa...",NaN,https://maps.google.com/?cid=89227966896180473...,NaN,...,True,True,True,False,True,True,True,True,NaN,NaN
2,places/ChIJBYSoMRIR2jERE2qfy1xjvdM,ChIJBYSoMRIR2jERE2qfy1xjvdM,"[indian_restaurant, restaurant, food, point_of...",NaN,NaN,"Singapore 809939921, Singapore","[{'longText': 'Singapore 809939921', 'shortTex...",NaN,https://maps.google.com/?cid=15257460362805471...,NaN,...,True,True,True,False,True,True,NaN,True,True,NaN
3,places/ChIJz7DMF1sR2jERyj0DZ_hR0DM,ChIJz7DMF1sR2jERyj0DZ_hR0DM,"[indian_restaurant, restaurant, food, point_of...",NaN,NaN,Singapore,"[{'longText': 'Singapore', 'shortText': 'SG', ...",NaN,https://maps.google.com/?cid=37335742184121461...,NaN,...,True,True,True,False,True,NaN,True,NaN,NaN,NaN
4,places/ChIJgSewkZQR2jERmKbPOQmUcAE,ChIJgSewkZQR2jERmKbPOQmUcAE,"[restaurant, food, point_of_interest, establis...",6789 6789,+65 6789 6789,"13 mos, test street, Singapore 123456","[{'longText': 'test street', 'shortText': 'tes...",NaN,https://maps.google.com/?cid=10374555877504783...,NaN,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
5,places/ChIJ_VzKVgAR2jERPuiRf7izOpY,ChIJ_VzKVgAR2jERPuiRf7izOpY,"[restaurant, food, point_of_interest, establis...",NaN,NaN,Singapore,"[{'longText': 'Singapore', 'shortText': 'SG', ...",NaN,https://maps.google.com/?cid=10825162259334031...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,places/ChIJFY2_SSAR2jERGBbkP5Z5Qr8,ChIJFY2_SSAR2jERGBbkP5Z5Qr8,"[fast_food_restaurant, restaurant, food, point...",NaN,NaN,"Lentor Modern, Singapore 788887","[{'longText': 'Lentor Modern', 'shortText': 'L...",NaN,https://maps.google.com/?cid=13781711495931106...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,places/ChIJGQAlHQAR2jERhCY89XPmM1I,ChIJGQAlHQAR2jERhCY89XPmM1I,"[restaurant, food, point_of_interest, establis...",NaN,NaN,Singapore,"[{'longText': 'Singapore', 'shortText': 'SG', ...",NaN,https://maps.google.com/?cid=59233313206323216...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,places/ChIJjWzqWQAR2jERZJNw4-sg_Bc,ChIJjWzqWQAR2jERZJNw4-sg_Bc,"[restaurant, food, point_of_interest, establis...",NaN,NaN,Singapore,"[{'longText': 'Singapore', 'shortText': 'SG', ...",NaN,https://maps.google.com/?cid=17282925545086370...,NaN,...,NaN,True,NaN,False,NaN,True,NaN,NaN,NaN,NaN
9,places/ChIJMakes-0R2jERiCtvDVOLWhg,ChIJMakes-0R2jERiCtvDVOLWhg,"[restaurant, food, point_of_interest, establis...",28 287 417,+371 28 287 417,"88 Zion Rd, Egg Harbor Twp, Singapore 160088","[{'longText': 'Egg Harbor Twp', 'shortText': '...",1.0,https://maps.google.com/?cid=17548681936381653...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# place details api

fieldmask_3 = '*'
id = 'ChIJRywE--sZ2jERFLd3yhyGxUQ'   # newtown food centre

test_3 = call_place_details_api(fieldmask_2, id, api_key)

In [74]:
test_3

,name,id,types,formattedAddress,addressComponents,googleMapsUri,utcOffsetMinutes,adrFormatAddress,iconMaskBaseUri,iconBackgroundColor,...,displayName.languageCode,addressDescriptor.landmarks,addressDescriptor.areas,googleMapsLinks.directionsUri,googleMapsLinks.placeUri,googleMapsLinks.photosUri,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.addressLines
0,places/ChIJRywE--sZ2jERFLd3yhyGxUQ,ChIJRywE--sZ2jERFLd3yhyGxUQ,"[premise, street_address]","Newton, Newton Food Centre, Singapore","[{'longText': 'Newton Food Centre', 'shortText...",https://maps.google.com/?cid=49555144232051484...,480,"<span class=""street-address"">Newton</span>, Ne...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,...,en,[{'name': 'places/ChIJ6wSMeewZ2jERnRkQnVf_7-c'...,[{'name': 'places/ChIJRywE--sZ2jERFLd3yhyGxUQ'...,https://www.google.com/maps/dir//''/data=!4m7!...,https://maps.google.com/?cid=49555144232051484...,https://www.google.com/maps/place//data=!4m3!3...,Asia/Singapore,SG,en-US,[Newton]
